In [1]:
import shutil
import random
from pathlib import Path

In [2]:
base_ndvi = Path("/mnt/sdb-seagate/graduacao/datasets/projeto_cerrado/fotos-rotuladas/train_ndvi")
base_ndre = Path("/mnt/sdb-seagate/graduacao/datasets/projeto_cerrado/fotos-rotuladas/train_ndre")

src_ndvi_imgs = base_ndvi / "images"
src_ndvi_lbls = base_ndvi / "labels"

src_ndre_imgs = base_ndre / "images"
src_ndre_lbls = base_ndre / "labels"

out_base = Path("/mnt/sdb-seagate/graduacao/datasets/projeto_cerrado/datasets_finais")

In [3]:
def criar_pastas(path_dataset):
    """Cria a estrutura train/val/test dentro do dataset alvo"""
    for split in ['train', 'val', 'test']:
        (path_dataset / split / 'images').mkdir(parents=True, exist_ok=True)
        (path_dataset / split / 'labels').mkdir(parents=True, exist_ok=True)

In [4]:
def copiar_arquivos(id_arquivo_ndvi, split, tipo_indice):
    """
    Copia o par (imagem + label).
    id_arquivo_ndvi: O nome do arquivo (sem extensão) vindo da pasta NDVI.
    """
    
    # 1. Definir pastas de origem e destino
    if tipo_indice == 'ndvi':
        folder_img = src_ndvi_imgs
        folder_lbl = src_ndvi_lbls
        dst_root = out_base / "dataset_ndvi"
        
        # Para NDVI, o nome é o próprio ID que achamos na lista
        nome_busca = id_arquivo_ndvi
        
    else: # NDRE
        folder_img = src_ndre_imgs
        folder_lbl = src_ndre_lbls
        dst_root = out_base / "dataset_ndre"
        
        # --- AQUI ESTAVA O ERRO ---
        # Temos que transformar o nome do NDVI no nome do NDRE
        # Ex: "foto_01_NDVI" vira "foto_01_NDRE"
        if "_NDVI" in id_arquivo_ndvi:
            nome_busca = id_arquivo_ndvi.replace("_NDVI", "_NDRE")
        elif "_ndvi" in id_arquivo_ndvi:
            nome_busca = id_arquivo_ndvi.replace("_ndvi", "_NDRE") # Força maiúsculo pois seu ls mostrou NDRE
        else:
            # Se não tiver o sufixo, tenta adicionar (ou manter igual)
            nome_busca = id_arquivo_ndvi.replace("_RGB", "_RGB_NDRE") 

    # 2. Achar a imagem (independente de ser .tif ou .TIF)
    # Procura por nome_busca + extensão
    candidatos = list(folder_img.glob(f"{nome_busca}.[tT][iI][fF]*"))
    
    if not candidatos:
        # Tenta uma busca mais agressiva se não achou (as vezes o sufixo muda)
        if tipo_indice == 'ndre':
            # Tenta achar apenas pelo número do ID (ex: 0061) se falhar o nome exato
            parts = id_arquivo_ndvi.split('_')
            for part in parts:
                if part.isdigit() and len(part) == 4: # Se achar "0061"
                    candidatos = list(folder_img.glob(f"*{part}*NDRE.[tT][iI][fF]*"))
                    break
    
    if not candidatos:
        print(f"⚠️ [{tipo_indice.upper()}] Imagem não encontrada para: {nome_busca}")
        return

    src_img_path = candidatos[0]
    
    # O Label geralmente tem o mesmo nome da imagem encontrada
    src_lbl_path = folder_lbl / f"{src_img_path.stem}.txt"
    
    # 3. Verificações de Segurança
    if not src_lbl_path.exists():
        print(f"⚠️ [{tipo_indice.upper()}] Label não existe: {src_lbl_path.name}")
        return

    # 4. Copiar para o DESTINO
    # Forçamos a saída para .tif minúsculo para padronizar
    shutil.copy(src_img_path, dst_root / split / 'images' / f"{src_img_path.stem}.tif")
    shutil.copy(src_lbl_path, dst_root / split / 'labels' / src_lbl_path.name)

In [5]:
print("🚀 Iniciando Separação Espelhada (NDVI + NDRE)...")

# Limpa output antigo para não misturar dados
if out_base.exists():
    print("🧹 Limpando pasta de destino antiga...")
    shutil.rmtree(out_base)

# Prepara as pastas de destino
path_ndvi_out = out_base / "dataset_ndvi"
path_ndre_out = out_base / "dataset_ndre"
criar_pastas(path_ndvi_out)
criar_pastas(path_ndre_out)

# 1. Listar IDs Únicos (Usando NDVI como referência mestre)
# Pega apenas o 'stem' (nome sem extensão)
arquivos_ndvi = list(src_ndvi_imgs.glob("*.TIF"))

if not arquivos_ndvi:
    print("❌ ERRO CRÍTICO: Nenhuma imagem encontrada na pasta NDVI!")
    print(f"   Confira: {src_ndvi_imgs}")

ids_unicos = [f.stem for f in arquivos_ndvi]
print(f"📦 Total de amostras encontradas: {len(ids_unicos)}")

# 2. Embaralhar (SEED 42 = Sempre o mesmo resultado)
random.seed(42)
random.shuffle(ids_unicos)

# 3. Dividir Matemática (80% / 10% / 10%)
total = len(ids_unicos)
idx_train = int(total * 0.8)
idx_val = int(total * 0.9) # 80% + 10%

train_ids = ids_unicos[:idx_train]
val_ids = ids_unicos[idx_train:idx_val]
test_ids = ids_unicos[idx_val:]

print(f"📊 Distribuição: Train={len(train_ids)} | Val={len(val_ids)} | Test={len(test_ids)}")

# 4. Loop de Cópia
print("   Copiando arquivos... (Isso garante que o ID X vá para o mesmo split nos dois datasets)")

for id_ in train_ids:
    copiar_arquivos(id_, 'train', 'ndvi')
    copiar_arquivos(id_, 'train', 'ndre')
    
for id_ in val_ids:
    copiar_arquivos(id_, 'val', 'ndvi')
    copiar_arquivos(id_, 'val', 'ndre')
    
for id_ in test_ids:
    copiar_arquivos(id_, 'test', 'ndvi')
    copiar_arquivos(id_, 'test', 'ndre')

print("\n✅ Concluído com Sucesso!")
print(f"   NDVI pronto em: {path_ndvi_out}")
print(f"   NDRE pronto em: {path_ndre_out}")
print("👉 Não esqueça de copiar o 'data.yaml' para dentro dessas pastas!")

🚀 Iniciando Separação Espelhada (NDVI + NDRE)...
🧹 Limpando pasta de destino antiga...
📦 Total de amostras encontradas: 324
📊 Distribuição: Train=259 | Val=32 | Test=33
   Copiando arquivos... (Isso garante que o ID X vá para o mesmo split nos dois datasets)

✅ Concluído com Sucesso!
   NDVI pronto em: /mnt/sdb-seagate/graduacao/datasets/projeto_cerrado/datasets_finais/dataset_ndvi
   NDRE pronto em: /mnt/sdb-seagate/graduacao/datasets/projeto_cerrado/datasets_finais/dataset_ndre
👉 Não esqueça de copiar o 'data.yaml' para dentro dessas pastas!
